In [ ]:
#@title 🎧 Download Narration Audio & Play Introduction
import os as _os
if not _os.path.exists("/content/narration"):
    !pip install -q gdown
    import gdown
    gdown.download(id="1BTEmYxTT95GZ05nMjwvaGWt0KwF-7ifD", output="/content/narration.zip", quiet=False)
    !unzip -q /content/narration.zip -d /content/narration
    !rm /content/narration.zip
    print(f"Loaded {len(_os.listdir('/content/narration'))} narration segments")
else:
    print("Narration audio already loaded.")

from IPython.display import Audio, display
display(Audio("/content/narration/00_intro.mp3"))

In [ ]:
# 🔧 Setup: Run this cell first!
# Check GPU availability and install dependencies

import torch
import sys

# Check GPU
if torch.cuda.is_available():
    device = torch.device('cuda')
    print(f"✅ GPU available: {torch.cuda.get_device_name(0)}")
    print(f"   Memory: {torch.cuda.get_device_properties(0).total_memory / 1e9:.1f} GB")
else:
    device = torch.device('cpu')
    print("⚠️ No GPU detected. Some cells may run slowly.")
    print("   Go to Runtime → Change runtime type → GPU")

print(f"\n📦 Python {sys.version.split()[0]}")
print(f"🔥 PyTorch {torch.__version__}")

# Set random seeds for reproducibility
import random
import numpy as np

SEED = 42
random.seed(SEED)
np.random.seed(SEED)
torch.manual_seed(SEED)
if torch.cuda.is_available():
    torch.cuda.manual_seed_all(SEED)

print(f"🎲 Random seed set to {SEED}")

%matplotlib inline

# 🚀 VAE: Compressing the Visual World from First Principles

*Part 1 of the Vizuara series on World Models*
*Estimated time: 45 minutes*

# 🤖 AI Teaching Assistant

Need help with this notebook? Open the **AI Teaching Assistant** — it has already read this entire notebook and can help with concepts, code, and exercises.

**[👉 Open AI Teaching Assistant](https://pods.vizuara.ai/courses/world-models/practice/1/assistant)**

*Tip: Open it in a separate tab and work through this notebook side-by-side.*


In [ ]:
#@title 🎧 Listen: Why It Matters
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/02_why_it_matters.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 1. Why Does This Matter?

Imagine you are building an AI agent that learns to drive a car in a video game. At every moment, the agent sees a 64×64 pixel image — that is 12,288 numbers (64 × 64 × 3 color channels). Trying to make decisions directly from 12,288 numbers is like trying to navigate a city by memorizing every single brick in every building. It is overwhelmingly detailed and computationally expensive.

What if we could teach the agent to "sketch" the scene instead? Capture just the *essence* — the curve of the road, the position of the car, the upcoming turn — in a tiny vector of just 32 numbers?

This is exactly what a **Variational Autoencoder (VAE)** does. It is the "Vision" component (V) of the World Model architecture, and it compresses raw images into a compact **latent code** that preserves everything the agent needs to know.

By the end of this notebook, you will:
- Build a VAE from scratch in PyTorch
- Train it on MNIST digits (a clean, fast testbed)
- Watch it reconstruct images from just 16 numbers
- Interpolate smoothly between digits in latent space — seeing how the VAE organizes its internal "sketch pad"

Let us start.

In [ ]:
#@title 🎧 Listen: Setup
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/01_setup.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# 🔧 Setup and GPU check
import torch
import torch.nn as nn
import torch.nn.functional as F
import torch.optim as optim
from torchvision import datasets, transforms
from torch.utils.data import DataLoader
import matplotlib.pyplot as plt
import numpy as np

%matplotlib inline

device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Using device: {device}")

# Reproducibility
torch.manual_seed(42)
np.random.seed(42)

In [ ]:
#@title 🎧 Listen: Building Intuition
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/03_building_intuition.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 2. Building Intuition

### The Core Idea: Compression and Reconstruction

Think about how a talented sketch artist works. Someone shows them a detailed photograph, and in seconds they capture the essence in a quick pencil sketch — the overall shape, the key features, the spatial relationships. That sketch is dramatically simpler than the photograph, but it contains all the information needed to understand the scene.

A VAE works the same way:
1. The **Encoder** is the sketch artist — it looks at the full image and produces a tiny "sketch" (the latent code)
2. The **Decoder** is like an artist who reconstructs the full image from just the sketch

But here is what makes a VAE special compared to a regular autoencoder: the encoder does not output a single fixed sketch. Instead, it outputs a *probability distribution* — a cloud of possible sketches.

### 🤔 Think About This

Why would we want the encoder to output a distribution instead of a single point?

Hint: Think about what happens when you try to compress similar but slightly different images. A photo of the digit "7" written with a slight lean versus a perfectly vertical "7" — should they map to the exact same latent code?

The answer is no. By outputting a distribution, the VAE acknowledges that compression is inherently uncertain. Slightly different inputs can map to overlapping regions in latent space, which means the latent space becomes *smooth* — nearby points decode to similar images. This smoothness is what allows us to interpolate between images later.

In [ ]:
#@title 🎧 Listen: Mathematics
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/04_mathematics.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 3. The Mathematics

### The Reparameterization Trick

The encoder outputs two vectors for each input image: a **mean** $\mu$ and a **log-variance** $\log \sigma^2$. To get the latent code $z$, we sample from this distribution:

$$z = \mu + \sigma \cdot \epsilon, \quad \epsilon \sim \mathcal{N}(0, 1)$$

This equation says: take the mean $\mu$, then add some noise scaled by $\sigma$. The noise $\epsilon$ is sampled from a standard normal distribution.

Computationally: if $\mu = 0.5$ and $\sigma = 0.2$, and we happen to sample $\epsilon = 0.3$:

$$z = 0.5 + 0.2 \times 0.3 = 0.56$$

Why this trick? During training, we need gradients to flow through the sampling operation. We cannot backpropagate through a random sample. But by rewriting the sample as $\mu + \sigma \cdot \epsilon$, the randomness is isolated in $\epsilon$ (which does not depend on any parameters), and gradients flow cleanly through $\mu$ and $\sigma$.

### The VAE Loss Function

The VAE is trained with two objectives:

$$\mathcal{L}_{\text{VAE}} = \underbrace{\text{BCE}(x, \hat{x})}_{\text{Reconstruction Loss}} + \underbrace{D_{\text{KL}}(q(z|x) \| \mathcal{N}(0, 1))}_{\text{KL Divergence}}$$

- **Reconstruction loss**: How well does the decoded image $\hat{x}$ match the original $x$? We use binary cross-entropy (BCE) for this.
- **KL divergence**: How far is the encoder's distribution from a standard normal? This regularizer keeps the latent space well-organized.

The KL divergence has a beautiful closed-form solution for Gaussians:

$$D_{\text{KL}} = -\frac{1}{2} \sum_{j=1}^{d} \left(1 + \log \sigma_j^2 - \mu_j^2 - \sigma_j^2\right)$$

Computationally: for each dimension $j$ of the latent code, we compute this term using $\mu_j$ and $\log \sigma_j^2$ — both of which the encoder directly outputs. No sampling needed!

In [ ]:
#@title 🎧 Listen: Encoder
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/05_encoder.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 4. Let's Build It — Component by Component

### 4.1 The Encoder

The encoder takes a 28×28 grayscale image (MNIST) and compresses it into parameters of a distribution in latent space.

In [ ]:
LATENT_DIM = 16  # We will compress 784 pixels into just 16 numbers!

class Encoder(nn.Module):
    def __init__(self, latent_dim=LATENT_DIM):
        super().__init__()
        # 28x28 = 784 -> 400 -> latent_dim
        self.fc1 = nn.Linear(784, 400)
        self.fc_mu = nn.Linear(400, latent_dim)      # Mean of the distribution
        self.fc_logvar = nn.Linear(400, latent_dim)   # Log-variance of the distribution

    def forward(self, x):
        # Flatten the image: (batch, 1, 28, 28) -> (batch, 784)
        x = x.view(x.size(0), -1)
        h = F.relu(self.fc1(x))
        mu = self.fc_mu(h)
        logvar = self.fc_logvar(h)
        return mu, logvar

# Test the encoder
encoder = Encoder().to(device)
dummy_input = torch.randn(4, 1, 28, 28).to(device)
mu, logvar = encoder(dummy_input)
print(f"Input shape:   {dummy_input.shape}")
print(f"mu shape:      {mu.shape}")
print(f"logvar shape:  {logvar.shape}")
print(f"Compression:   784 values -> {LATENT_DIM} (mu) + {LATENT_DIM} (logvar)")

Notice how the encoder outputs TWO vectors of size 16 — the mean and the log-variance. These parameterize a 16-dimensional Gaussian distribution in latent space.

### 4.2 The Reparameterization Trick

This is the clever bit that makes VAE training possible. We need to sample from the encoder's distribution, but in a way that allows gradients to flow.

In [ ]:
#@title 🎧 Listen: Reparam And Decoder
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/06_reparam_and_decoder.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
def reparameterize(mu, logvar):
    """
    Sample z from q(z|x) using the reparameterization trick.

    Instead of sampling z ~ N(mu, sigma^2) directly,
    we sample epsilon ~ N(0, 1) and compute z = mu + sigma * epsilon.
    This lets gradients flow through mu and sigma.
    """
    std = torch.exp(0.5 * logvar)   # sigma = exp(0.5 * log(sigma^2))
    eps = torch.randn_like(std)      # Sample from N(0, 1)
    z = mu + std * eps               # The trick!
    return z

# Test it
z = reparameterize(mu, logvar)
print(f"Sampled z shape: {z.shape}")
print(f"First sample z[0]: {z[0].detach().cpu().numpy().round(3)}")

# Sample again — different z because epsilon is random!
z2 = reparameterize(mu, logvar)
print(f"Second sample z[0]: {z2.detach().cpu().numpy().round(3)}")
print(f"Same mu, different samples — this is the stochasticity!")

### 4.3 The Decoder

The decoder takes a latent code $z$ and reconstructs the original image.

In [ ]:
class Decoder(nn.Module):
    def __init__(self, latent_dim=LATENT_DIM):
        super().__init__()
        # latent_dim -> 400 -> 784
        self.fc1 = nn.Linear(latent_dim, 400)
        self.fc2 = nn.Linear(400, 784)

    def forward(self, z):
        h = F.relu(self.fc1(z))
        # Sigmoid to get pixel values in [0, 1]
        reconstruction = torch.sigmoid(self.fc2(h))
        return reconstruction.view(-1, 1, 28, 28)

# Test the decoder
decoder = Decoder().to(device)
recon = decoder(z)
print(f"Latent z shape:       {z.shape}")
print(f"Reconstruction shape: {recon.shape}")

In [ ]:
#@title 🎧 Listen: Before Training Viz
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/07_before_training_viz.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

In [ ]:
# 📊 Visualization: What does random decoding look like before training?
fig, axes = plt.subplots(1, 8, figsize=(16, 2))
for i in range(8):
    random_z = torch.randn(1, LATENT_DIM).to(device)
    with torch.no_grad():
        img = decoder(random_z).cpu().squeeze()
    axes[i].imshow(img, cmap='gray')
    axes[i].axis('off')
    axes[i].set_title(f'z sample {i+1}', fontsize=9)
plt.suptitle('Decoded random latent codes (BEFORE training) — just noise!', fontsize=12)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Complete Vae
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/08_complete_vae.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

### 4.4 The Complete VAE

Now let us put encoder, reparameterization, and decoder together.

In [ ]:
class VAE(nn.Module):
    def __init__(self, latent_dim=LATENT_DIM):
        super().__init__()
        self.encoder = Encoder(latent_dim)
        self.decoder = Decoder(latent_dim)

    def reparameterize(self, mu, logvar):
        std = torch.exp(0.5 * logvar)
        eps = torch.randn_like(std)
        return mu + std * eps

    def forward(self, x):
        mu, logvar = self.encoder(x)
        z = self.reparameterize(mu, logvar)
        reconstruction = self.decoder(z)
        return reconstruction, mu, logvar

vae = VAE().to(device)
total_params = sum(p.numel() for p in vae.parameters())
print(f"Total VAE parameters: {total_params:,}")
print(f"Compression ratio: 784 / {LATENT_DIM} = {784/LATENT_DIM:.0f}x")

In [ ]:
#@title 🎧 Listen: Todo Loss
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/09_todo_loss.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 5. 🔧 Your Turn: Implement the VAE Loss Function

Now it is your turn. Implement the VAE loss — the sum of reconstruction loss and KL divergence.

In [ ]:
def vae_loss(reconstruction, original, mu, logvar):
    """
    Compute the VAE loss = Reconstruction Loss + KL Divergence.

    Args:
        reconstruction: Decoded image, shape (batch, 1, 28, 28)
        original: Original image, shape (batch, 1, 28, 28)
        mu: Encoder mean, shape (batch, latent_dim)
        logvar: Encoder log-variance, shape (batch, latent_dim)

    Returns:
        Total loss (scalar), reconstruction loss (scalar), KL loss (scalar)
    """
    # ============ TODO ============
    # Step 1: Compute reconstruction loss using F.binary_cross_entropy
    #         Remember to set reduction='sum' and flatten the images
    #         Hint: F.binary_cross_entropy(recon.view(-1, 784), orig.view(-1, 784), reduction='sum')
    #
    # Step 2: Compute KL divergence using the closed-form formula:
    #         KL = -0.5 * sum(1 + logvar - mu^2 - exp(logvar))
    #
    # Step 3: Return the sum of both losses
    # ==============================

    recon_loss = ???  # YOUR CODE HERE
    kl_loss = ???     # YOUR CODE HERE

    return recon_loss + kl_loss, recon_loss, kl_loss

In [ ]:
# ✅ Verification: Test your loss function
test_recon = torch.sigmoid(torch.randn(2, 1, 28, 28)).to(device)
test_orig = torch.rand(2, 1, 28, 28).to(device)
test_mu = torch.randn(2, LATENT_DIM).to(device)
test_logvar = torch.zeros(2, LATENT_DIM).to(device)  # sigma=1 -> KL should be 0

total, recon, kl = vae_loss(test_recon, test_orig, test_mu, test_logvar)

# When logvar=0 and mu!=0, KL = 0.5 * sum(mu^2)
expected_kl = 0.5 * test_mu.pow(2).sum()
assert torch.allclose(kl, expected_kl, atol=1e-4), \
    f"❌ KL divergence incorrect. Expected {expected_kl.item():.2f}, got {kl.item():.2f}"
assert recon.item() > 0, "❌ Reconstruction loss should be positive"
print(f"✅ Loss function works!")
print(f"   Reconstruction loss: {recon.item():.2f}")
print(f"   KL divergence: {kl.item():.2f}")
print(f"   Total loss: {total.item():.2f}")

In [ ]:
#@title 🎧 Listen: Stop And Think
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/10_stop_and_think.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

---

### ✋ Stop and Think

Before we train, predict:
1. What will happen to the reconstruction loss over training? (It should decrease — images get sharper)
2. What will happen to the KL loss? (It starts near 0, then increases as the encoder learns, then stabilizes)
3. After training, if we sample random $z \sim \mathcal{N}(0,1)$ and decode, what will we see? (Recognizable digits!)

*Take a moment. Then scroll down.*

---

In [ ]:
#@title 🎧 Listen: Training
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/11_training.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 6. Training the VAE

In [ ]:
# Load MNIST
transform = transforms.Compose([transforms.ToTensor()])
train_dataset = datasets.MNIST('./data', train=True, download=True, transform=transform)
test_dataset = datasets.MNIST('./data', train=False, transform=transform)

train_loader = DataLoader(train_dataset, batch_size=128, shuffle=True)
test_loader = DataLoader(test_dataset, batch_size=128, shuffle=False)

print(f"Training samples: {len(train_dataset)}")
print(f"Test samples: {len(test_dataset)}")
print(f"Image shape: {train_dataset[0][0].shape}")

In [ ]:
# Training loop
vae = VAE().to(device)
optimizer = optim.Adam(vae.parameters(), lr=1e-3)

num_epochs = 15
train_losses = []
recon_losses = []
kl_losses = []

for epoch in range(num_epochs):
    vae.train()
    epoch_loss = 0
    epoch_recon = 0
    epoch_kl = 0

    for batch_idx, (data, _) in enumerate(train_loader):
        data = data.to(device)
        optimizer.zero_grad()

        recon, mu, logvar = vae(data)
        loss, recon_loss, kl_loss = vae_loss(recon, data, mu, logvar)

        loss.backward()
        optimizer.step()

        epoch_loss += loss.item()
        epoch_recon += recon_loss.item()
        epoch_kl += kl_loss.item()

    # Average over all samples
    n = len(train_loader.dataset)
    train_losses.append(epoch_loss / n)
    recon_losses.append(epoch_recon / n)
    kl_losses.append(epoch_kl / n)

    print(f"Epoch {epoch+1:2d}/{num_epochs} | "
          f"Loss: {epoch_loss/n:.2f} | "
          f"Recon: {epoch_recon/n:.2f} | "
          f"KL: {epoch_kl/n:.2f}")

In [ ]:
# 📊 Training curves
fig, (ax1, ax2) = plt.subplots(1, 2, figsize=(14, 5))

ax1.plot(range(1, num_epochs+1), train_losses, 'b-', linewidth=2, label='Total Loss')
ax1.plot(range(1, num_epochs+1), recon_losses, 'r--', linewidth=2, label='Reconstruction')
ax1.plot(range(1, num_epochs+1), kl_losses, 'g--', linewidth=2, label='KL Divergence')
ax1.set_xlabel('Epoch', fontsize=12)
ax1.set_ylabel('Loss (per sample)', fontsize=12)
ax1.set_title('VAE Training Curves', fontsize=14)
ax1.legend(fontsize=11)
ax1.grid(True, alpha=0.3)

# Ratio of losses
ratios = [r / (r + k) for r, k in zip(recon_losses, kl_losses)]
ax2.plot(range(1, num_epochs+1), ratios, 'purple', linewidth=2)
ax2.set_xlabel('Epoch', fontsize=12)
ax2.set_ylabel('Recon / Total Loss', fontsize=12)
ax2.set_title('Reconstruction vs KL Balance', fontsize=14)
ax2.set_ylim(0, 1)
ax2.grid(True, alpha=0.3)

plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Results
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/12_results.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 7. Seeing the Results

### 7.1 Original vs. Reconstructed

In [ ]:
# 📊 Compare originals and reconstructions
vae.eval()
with torch.no_grad():
    test_data = next(iter(test_loader))[0][:10].to(device)
    recon, _, _ = vae(test_data)

fig, axes = plt.subplots(2, 10, figsize=(20, 4))
for i in range(10):
    axes[0, i].imshow(test_data[i].cpu().squeeze(), cmap='gray')
    axes[0, i].axis('off')
    axes[1, i].imshow(recon[i].cpu().squeeze(), cmap='gray')
    axes[1, i].axis('off')
axes[0, 0].set_ylabel('Original', fontsize=12, rotation=0, labelpad=60)
axes[1, 0].set_ylabel('Reconstructed', fontsize=12, rotation=0, labelpad=60)
plt.suptitle('VAE Reconstruction: 784 pixels → 16 latent numbers → 784 pixels', fontsize=14)
plt.tight_layout()
plt.show()

### 7.2 Generating New Digits from Random Latent Codes

In [ ]:
# 📊 Sample random z from N(0,1) and decode — the VAE generates new digits!
vae.eval()
fig, axes = plt.subplots(2, 10, figsize=(20, 4))
for i in range(20):
    row, col = i // 10, i % 10
    random_z = torch.randn(1, LATENT_DIM).to(device)
    with torch.no_grad():
        generated = vae.decoder(random_z).cpu().squeeze()
    axes[row, col].imshow(generated, cmap='gray')
    axes[row, col].axis('off')
plt.suptitle('Generated digits from random z ~ N(0,1) — the VAE has learned to dream!', fontsize=14)
plt.tight_layout()
plt.show()

In [ ]:
#@title 🎧 Listen: Interpolation
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/13_interpolation.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 8. 🎯 Final Output: Latent Space Interpolation

This is the moment that makes it all click. We will pick two digits, encode them to latent space, and smoothly interpolate between them. If the VAE has learned a good latent space, we should see a smooth morphing from one digit to another.

In [ ]:
# Find specific digits for interpolation
def get_digit(dataset, digit, index=0):
    """Get the index-th example of a specific digit."""
    count = 0
    for img, label in dataset:
        if label == digit:
            if count == index:
                return img.unsqueeze(0)
            count += 1
    return None

vae.eval()
pairs = [(1, 7), (3, 8), (4, 9), (2, 0)]
n_steps = 10

fig, axes = plt.subplots(len(pairs), n_steps, figsize=(20, 2 * len(pairs)))

for row, (d1, d2) in enumerate(pairs):
    img1 = get_digit(test_dataset, d1).to(device)
    img2 = get_digit(test_dataset, d2).to(device)

    with torch.no_grad():
        mu1, _ = vae.encoder(img1)
        mu2, _ = vae.encoder(img2)

        for col, alpha in enumerate(np.linspace(0, 1, n_steps)):
            z_interp = (1 - alpha) * mu1 + alpha * mu2
            generated = vae.decoder(z_interp).cpu().squeeze()
            axes[row, col].imshow(generated, cmap='gray')
            axes[row, col].axis('off')

    axes[row, 0].set_ylabel(f'{d1} → {d2}', fontsize=11, rotation=0, labelpad=40)

plt.suptitle('Latent Space Interpolation — Smooth Morphing Between Digits!', fontsize=14, y=1.02)
plt.tight_layout()
plt.show()
print("🎉 The VAE has learned a smooth, organized latent space!")
print("   This is exactly the kind of representation a World Model needs.")
print("   Next: we will learn to PREDICT how this latent space evolves over time.")

In [ ]:
#@title 🎧 Listen: Closing
from IPython.display import Audio, display
import os as _os
_f = "/content/narration/14_closing.mp3"
if _os.path.exists(_f):
    display(Audio(_f))
else:
    print("Run the first cell to download narration audio.")

## 9. Reflection and Next Steps

### 🤔 Reflection Questions
1. What happens if you increase the latent dimension from 16 to 64? Will reconstructions improve? What about interpolation?
2. What happens if you remove the KL term entirely (set it to 0)? This gives you a regular autoencoder — will interpolation still work smoothly?
3. Why is the KL term important for World Models specifically? (Hint: think about what the Memory component needs to do with the latent codes)

### 🏆 Optional Challenges
1. **Convolutional VAE**: Replace the fully-connected layers with convolutional layers. Do the reconstructions get sharper?
2. **Conditional generation**: Modify the decoder to also take a digit label as input. Can you generate any digit on demand?
3. **Beta-VAE**: Multiply the KL term by a factor $\beta > 1$. How does this affect the latent space organization?

### What is Next?

We have built the "eyes" of our World Model — the VAE that compresses visual observations into a compact latent code. But seeing is not enough. The agent also needs to *predict* what will happen next. In the next notebook, we will build the **MDN-RNN** — the Memory component that learns the dynamics of the world in latent space.